# 파이썬으로 FTP 클라이언트 만들기

* 난이도 : ★★★☆☆☆☆☆☆☆
* 필요라이브러리: ftplib, tqdm, os, sys


* FTP 서버에 접속하여 기능을 수행할 FTP 클라이언트를 만듭니다.
* FTP 기능을 활용하기 위해 ftplib 라이브러리를 사용합니다.
> pip install ftplib  
* 다운로드 상태를 프로그래스바로 출력하기 위해 tqdm 라이브러리를 사용합니다.
> pip install tqdm

## FTP 명령어
* FTP 명령어에 해당하는 명령어를 도스 명령 스타일로 변경하여 기능을 수행하게 합니다.

<table width="600">
    <tr>
        <td><center>명령어</center></td>
        <td><center>구현될 명령어</center></td>
        <td><center>설명</center></td>
    </tr>
    <tr>
        <td><p>nlist()</p></td>
        <td><p>dir, ls</p></td>
        <td><p>현재 경로의 폴더/파일 목록을 구합니다.</p></td>
    </tr>
    <tr>
        <td><p>cwd(path)</p></td>
        <td><p>cd</p></td>
        <td><p>FTP의 폴더 경로를 변경합니다.</p></td>
    </tr>
    <tr>
        <td><p>mkd(path)</p></td>
        <td><p>mkdir</p></td>
        <td><p>FTP 에 폴더를 생성합니다.</p></td>
    </tr>
    <tr>
        <td><p>delete(filename)</p></td>
        <td><p>delete, del</p></td>
        <td><p>FTP 에서 파일을 삭제합니다.</p></td>
    </tr>
    <tr>
        <td><p>retbinary('RETR ' + path)</p></td>
        <td><p>down [filepath]</p></td>
        <td><p>FTP 에서 파일을 바이너리 형태로 다운로드 합니다.</p></td>
    </tr>
    <tr>
        <td><p>storbinary('STOR ' + path)</p></td>
        <td><p>up [filepath]</p></td>
        <td><p>FTP 에 파일을 업로드 합니다.</p></td>
    </tr>
</table>

In [ ]:
# FTP 라이브러리
import ftplib
# 다운로드, 업로드시 progressbar 를 구현하기 위한 라이브러리
from tqdm import tqdm
import os
import sys

# FTP 접속 정보
FTP_SERVER = "localhost"
FTP_PORT = 11000
FTP_ID = "Administrator"
FTP_PASS = "remember0926"

def is_file(ftp, filename):
    '''파일인지 확인하는 함수
    
    Args:
        ftp (ftplib) : ftplib 접속 컨넥션
        filename (str) : 파일인지 확인할 파일명
    
    Returns:
        bool : True or False
    '''

    # 현재 경로를 임시로 저장해놓습니다.
    current = ftp.pwd()
    try:
        # 파일명으로 이동했을시 파일인 경우 오류가 발생하여 
        # except 가 수행됩니다.
        ftp.cwd(filename)
    except:
        # 원래 경로로 변경해놓고 True 리턴
        ftp.cwd(current)
        return True

    # 원래 경로로 변경 후 False (파일 아님) 리턴
    ftp.cwd(current)
    return False


# FTP 객체 생성
ftp = ftplib.FTP()

# 서버의 인코딩 상태 설정
ftp.encoding='euc-kr'

# FTP 서버 접속
ftp.connect(FTP_SERVER, FTP_PORT)

# FTP 서버 로그인
print(ftp.login(FTP_ID, FTP_PASS))

# 종료 명령 전까진 무한 반복
while True:
    # FTP 의 현재 경로
    cur_dir = ftp.pwd()

    # 사용자에게 명령프롬프트를 띄웁니다.
    # FTP /> 이런식..
    cmd = input("FTP {}> ".format(cur_dir))

    # 사용자에게 입력된 내용을 공백으로 분리하여 리스트화 시킵닏.
    args = cmd.split(" ")

    # 입력된 내용이 없으면 루프반복
    if len(args) <= 0:
        continue

    # 입력된 내용의 0 번째는 명령어로 간주 합니다.
    # ex) /down abcd.jpg
    command = args[0]

    # 입력된 명령어 삭제 후 아규먼트만 남깁니다.
    del args[0]

    # 명령어가 exit 면 while 문 탈출
    if command == "exit":
        break
    # 명령어가 dir 혹은 ls 면
    if command == "dir" or command == "ls":
        # 현재 FTP 경로의 파일 / 폴더목록 구함
        lists = ftp.nlst()

        # 목록 루프
        for l in lists:
            # 파일이면 그냥 출력 
            if is_file(ftp, l):
                print(l)
            # 폴더면 /test/ 이런식으로 출력
            else:
                print("{}{}/".format(cur_dir, l))
    # 이동명령이면
    elif command == "cd":
        target = args[0]
        # 파일이 아닌경우 이동
        if not is_file(ftp, target):
            ftp.cwd(target)
        else:
            print("대상이 파일 입니다.")
    # 폴더 생성 명령이면
    elif command == "mkdir":
        target = args[0]
        ftp.mkd(target)
    # 다운로드 명령이면
    elif command == "down":
        filepath = args[0]
        # 파일을 wb 모드로 열기 후
        with open(filepath, 'wb') as file:
            # 파일 사이즈 체크
            size = ftp.size(filepath)
            # 프로그래스바 생성
            with tqdm(unit='blocks', unit_scale=True, leave=False, miniters=1, desc='Uploading......', total=size) as tqdm_instance:
                # 다운로드시 호출되는 콜백함수
                def callback(data):
                    # 프로그래스바를 받은 데이터 크기만큼 업데이트
                    tqdm_instance.update(len(data))
                    # 로컬 파일로 저장
                    file.write(data)
                # ftp 다운로드 호출 콜백함수 지정!!!!
                ftp.retrbinary('RETR ' + filepath, callback=callback)
    # 업로드 명령이면
    elif command == "up":
        # 업로드 대상 로컬 파일 경로
        local_filepath = args[0]
        # 파일명만 분리
        filename = local_filepath.split("\\")[-1]
        
        # 업로드될 파일의 사이즈를 구합니다.
        size = os.path.getsize(local_filepath)
        # 파일 rb 모드로 열기
        with open(local_filepath, 'rb') as file:
            # 프로그래스바 생성
            with tqdm(unit='blocks', unit_scale=True, leave=False, miniters=1, desc='Uploading......', total=size) as tqdm_instance:
                #  FTP 업로드 호출, 콜백함수에 프로그래스바 업데이트 함수를 지정합니다.
                ftp.storbinary('STOR ' + filename, file, 2048, callback=lambda sent: tqdm_instance.update(len(sent)))
    # 파일삭제 명령시
    elif command == "delete" or command == "del":
        filename = args[0]
        ftp.delete(filename)

# 무한루프를 빠져나오면 ftp 를 종료 합니다.
ftp.close()
